### 1. import modules

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import pickle
import os
import json
from ylib import ylog
from tqdm import *
import matplotlib.pyplot as plt
import cx_Oracle as cx
from collections import defaultdict 
from pyquery import PyQuery as pq
from bs4 import BeautifulSoup

np.set_printoptions(suppress=True)
%precision %.5g
%matplotlib inline 
import logging
from gensim import corpora, models, similarities
from gensim.matutils import jaccard
from gensim.corpora.dictionary import Dictionary
from gensim.matutils import Sparse2Corpus

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
import ylib.preprocessing as prep
from gensim.models import LsiModel, LdaModel

/home/wuwei/anaconda3/envs/tfgpu/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


### 2. database connection configuration

In [2]:
tqdm.pandas()

In [3]:
os.environ['NLS_LANG'] = 'SIMPLIFIED CHINESE_CHINA.UTF8'
conn_kbms = cx.connect('risk_matrix/risk_matrix@192.168.4.30:1521/orcl')
conn_patient = cx.connect('MMAPV41/MMAPV411556@192.168.4.32:1521/orcl')
ylog.set_level(logging.DEBUG)
ylog.console_on()
ylog.filelog_on("topics")

### 3. load data

In [4]:
# 解析详情页
def parse_detail(html):
    doc = pq(html)
    title = doc('.rich_media_title').text()
    content = doc('.rich_media_content').text()
    wechat_name = doc('#js_profile_qrcode > div > strong').text()
    nickname = doc('.rich_media_meta_text').text()
    wechat = doc('#js_profile_qrcode > div > p:nth-child(3) > span').text()
    date = None
    return {
        'title':title,
        'wechat_name': wechat_name,
        'content':content,
        'date':date,
        'nickname':nickname,
        'wechat':wechat
    }


In [5]:
dir_path = './data/'

In [6]:
ls_articles = []
for root, dirs, files in os.walk(dir_path):
    print(root, "has", len(files))
    for name in files:
        file_path = os.path.join(root, name)
        with open(file_path) as fp:
            article_content = fp.read()
        article = parse_detail(article_content)
        ls_articles.append(article)
    # print(sum([os.path.getsize(os.path.join(root, name)) for name in files]), '\s')
    # print("bytes in", len(files), "non-directory files")


./data/ has 0
./data/40秒 has 10
./data/Chihiro Quantitative Research has 12
./data/PureDelta has 11
./data/cfuwxd has 11
./data/codeMakeMoney has 1
./data/万孟岩 has 15
./data/东京交易厅 has 12
./data/人民币交易与研究 has 49
./data/付鹏的财经世界 has 13
./data/全球资产观察 has 10
./data/关关楼西 has 10
./data/冲浪团 has 10
./data/冷眼局中人 has 14
./data/加州分析员 has 10
./data/半夏投资 has 10
./data/华尔街情报圈 has 17
./data/博策远略 has 20
./data/向小田科技观察 has 11
./data/外汇头条 has 22
./data/宏观大类资产配置研究 has 13
./data/定投十年赚十倍 has 50
./data/岭峰资本 has 10
./data/川流不息d has 13
./data/川流不息skl has 6
./data/市场观察家 has 10
./data/扑克投资家 has 188
./data/投研帮 has 34
./data/新全球资产配置 has 10
./data/方哥谈基 has 10
./data/月风投资笔记 has 10
./data/期权交易策略 has 10
./data/李超宏观研究与资产配置 has 21
./data/沧海一土狗 has 11
./data/深圳天谷资产管理有限公司 has 10
./data/淳臻投资 has 26
./data/混沌巡洋舰 has 16
./data/湖畔公寓28号 has 10
./data/私募工场 has 73
./data/筹码 has 10
./data/聚宽量化实验室 has 21
./data/聪明投资者 has 28
./data/量化投资与机器学习 has 30
./data/量化投资大家学 has 10
./data/金融学前沿论文速递 has 12
./data/长赢指数投资 has 11
./data/靠门三思 has 10


In [7]:
df_articles = pd.DataFrame.from_records(ls_articles)


In [8]:
df_articles['articles'] = df_articles['content'].progress_apply(lambda x: prep.preprocess_string(x, [prep.cut_paragraph, prep.tokenize, prep.strip_punctuation, prep.strip_numeric]))

  0%|          | 0/1033 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
2019-07-23 18:01:36,686 : DEBUG : Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
2019-07-23 18:01:37,320 : DEBUG : Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.690 seconds.
2019-07-23 18:01:37,377 : DEBUG : Loading model cost 0.690 seconds.
Prefix dict has been built succesfully.
2019-07-23 18:01:37,379 : DEBUG : Prefix dict has been built succesfully.
100%|██████████| 1033/1033 [00:12<00:00, 82.81it/s] 


In [9]:
df_articles['articles']

0       话 说 这 几天 本来 在 写 卡尔   伊坎   但 实在 是 被 一些 媒体 的 傻 X...
1       昨天 在   Institutional \n Investor   上 看到 一篇 文章 ...
2       这 一次   索罗斯 大爷 又 说 对 了   \n 在 脱欧 公投 前   他 就 在  ...
3       事先 声明   本人 和 文中 提及 的 商业机构 或 人物 均 无 利益 关系   \n ...
4       我 这 人 最 不 喜欢 转 别人 文章   但 今天 必须 要 破 一次 例   没 别的...
5       刚才 看 了 下 留言 和 评论   发现 很多 同学 看 了 昨天 那篇   都 被 伊隆...
6       这 是 我 前 几天 给 红杉 资本 写 的 一篇 约稿   感觉 内容 还 行   所以 ...
7       在 这个 星球 上   你 应该 很难 找到 比 尤 瑟夫   卡巴 杰   Youssef...
8       我 看 后台 有 很多 同学 嫌 我 发文 太 少 不够看   好 啊   今天 我 就 发...
9       很多 同学 留言 说 上次 聊 Ackman 那篇太长 了   那 我 今天 发篇 短 的 ...
10      工欲善其事   必先利其器   无论是 研究 团队 协作   还是 个人 独立 专研   如...
11      今天 给 大家 分享 一本 因子 投资 的 好书     Your \n Complete ...
12      更好 的 阅读 体验   可以 点击 阅读 原文 获取 PDF 版本   欢迎 转发 \n ...
13       月  日晚   上交所 披露 了  家科 创板 受理 企业   分别 为晶晨 半导体   ...
14      本篇 为 刀疤 连发 在   因子 动物园   上 的 文章   主要 介绍 低 风险 效应...
15      本篇 为 刀疤 连为   因子 动物园   写 的 第二篇 文章   主要 介绍 各类 低 ...
16      感谢 李洋   刘 洋溢 和 石川 博士 对 本文 提出 的 修改 建议   由于 本文 篇...
17      本文 转载自

In [10]:
ls_articles[6]

{'title': '驯服贝克汉姆和C罗的人',
 'wechat_name': '40秒',
 'content': '这是我前几天给红杉资本写的一篇约稿，感觉内容还行，所以就转过来啦。本文首发于红杉汇（id:Sequoiacap）\n\n\n\n\n即使你从来都不看球，也一定听说过弗格森这个名字。\n作为史上最传奇的足球教练之一，他老人家一手发掘和教出了坎通纳、C罗、小贝、吉格斯等一大批巨星，并把曼联从一支惨兮兮的保级队带成了全球最赚钱的球队。\n在1999年，他还带队拿到了史无前例的欧冠、英超和足总杯三冠王。英国女王闻讯龙颜大悦，立马封了他骑士爵位，从此江湖人称弗爵爷（SAF, Sir Alex Ferguson）。\n\n\n▲\xa0爵爷手里拿的这个勋章可是稀罕货，读作Knight Bachelor，每年只发几十个，只要拿了就是Sir。比小贝拿的OBE（Officer of the Order of the British Empire，每年发900个）强多了。\n\n\n\n然而，这些都不是他最牛逼的地方。\n弗爵爷真正的牛逼之处，是他做到了每个教练都想做、但从来没人能做到的事——将一支球队带到全球顶尖水平不难，难的是，还能保持几十年之久。在他执掌曼联的26年里，球队拿下了整整38个顶级联赛的冠军，堪称空前绝后。\n在每个时期，曼联都有不同的死对头，如1990年代的利物浦、阿森纳，2000年后的切尔西、曼城等；与弗爵爷对弈的高手也有温格、穆里尼奥这样的名帅。但最终，都是弗爵爷带领的红魔在不同时期一次次登顶英超冠军。\n\n\n▲\xa01999年曼联三冠王。无数次的逆境反击\n弗格森领导力法则1：\n为赢敢赌\n\n\n曼联似乎有一种本领，总会在比赛后期战胜对手，多次上演绝地反转。“当球迷看到曼联在比赛最后15分钟努力争胜的景象，这种感觉无与伦比：队员踢得真刀实枪、轮番轰炸、甚至‘横尸遍野’。当然，我们也可能会在对手的防守反击中败北，但反败为胜带来的欣喜胜过一切。”\n\n\n要知道，足球教练可是地球上竞争最残酷的职业之一。不管你水平多高，只要连输个几场球立马就得滚蛋。在任何一个顶级联赛，教练的执教时间都是以月来计算的。\n就拿“狂人”穆里尼奥来说吧。作为新生代的顶级名帅，他在短短十几年里就拿了二十几个冠军，但也照样被撵得东奔西跑，每隔两三年就得换工作。

### mapping

#### preview patient data

#### cut items

### 4. topic modeling

In [11]:
# 只从文本中提取1000个最重要的特征关键词，然后停止。
n_features = 1000

#### 关键词提取和向量转换

In [12]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                lowercase=False,
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(df_articles['articles'])

In [13]:
print(tf.shape)  # (NO_DOCUMENTS, NO_FEATURES)
print(tf)

(1033, 1000)
  (0, 213)	1
  (0, 73)	1
  (0, 617)	1
  (0, 138)	1
  (0, 565)	1
  (0, 912)	1
  (0, 360)	1
  (0, 892)	1
  (0, 941)	1
  (0, 999)	1
  (0, 20)	1
  (0, 22)	1
  (0, 302)	1
  (0, 190)	1
  (0, 370)	1
  (0, 64)	1
  (0, 318)	2
  (0, 762)	1
  (0, 274)	1
  (0, 89)	1
  (0, 119)	1
  (0, 963)	2
  (0, 259)	1
  (0, 218)	1
  (0, 272)	1
  :	:
  (1032, 318)	1
  (1032, 127)	12
  (1032, 452)	1
  (1032, 171)	1
  (1032, 69)	1
  (1032, 734)	1
  (1032, 700)	2
  (1032, 450)	1
  (1032, 207)	1
  (1032, 498)	2
  (1032, 430)	1
  (1032, 30)	1
  (1032, 921)	3
  (1032, 821)	6
  (1032, 334)	2
  (1032, 915)	1
  (1032, 26)	3
  (1032, 604)	1
  (1032, 184)	1
  (1032, 71)	1
  (1032, 580)	3
  (1032, 28)	1
  (1032, 923)	2
  (1032, 470)	3
  (1032, 16)	1


In [14]:
dictionary = corpora.Dictionary(tokenized_data)

NameError: name 'tokenized_data' is not defined

In [15]:
corpus_vect_gensim = Sparse2Corpus(tf, documents_columns=False)

In [16]:
lsi = LsiModel(corpus_vect_gensim, num_topics=4)

2019-07-23 18:02:04,267 : WARNING : no word id mapping provided; initializing from corpus, assuming identity
2019-07-23 18:02:04,372 : INFO : using serial LSI version on this node
2019-07-23 18:02:04,373 : INFO : updating model with new documents
2019-07-23 18:02:04,422 : INFO : preparing a new chunk of documents
2019-07-23 18:02:04,482 : INFO : using 100 extra samples and 2 power iterations
2019-07-23 18:02:04,484 : INFO : 1st phase: constructing (1000, 104) action matrix
2019-07-23 18:02:04,509 : INFO : orthonormalizing (1000, 104) action matrix
2019-07-23 18:02:04,615 : INFO : 2nd phase: running dense svd on (104, 1033) matrix
2019-07-23 18:02:04,631 : INFO : computing the final decomposition
2019-07-23 18:02:04,633 : INFO : keeping 4 factors (discarding 53.498% of energy spectrum)
2019-07-23 18:02:04,634 : INFO : processed documents up to #1033
2019-07-23 18:02:04,636 : INFO : topic #0(1512.272): 0.672*"360" + 0.232*"767" + 0.207*"189" + 0.182*"890" + 0.144*"697" + 0.136*"752" + 0.

In [17]:
dictionary = Dictionary.from_corpus(corpus_vect_gensim, id2word=dict((id, word) for word, id in tf_vectorizer.vocabulary_.items()))

2019-07-23 18:02:06,399 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-07-23 18:02:06,650 : INFO : built Dictionary(1000 unique tokens: ['一些', '媒体', '今天', '必须', '这种']...) from 1033 documents (total 557542 corpus positions)


In [18]:
# transform sparse matrix into gensim corpus
# corpus_vect_gensim = gensim.matutils.Sparse2Corpus(tf_vectorizer, documents_columns=False)
lsi = LsiModel(corpus_vect_gensim, num_topics=4)
# I instead would like something like this line below
# lsi = gensim.models.LsiModel(corpus_vect_gensim, id2word=vect.vocabulary_, num_topics=2)
print (lsi.print_topics(2))
#['0.622*"21" + 0.359*"31" + 0.256*"38" + 0.206*"29" + 0.206*"34" + 0.197*"36" + 0.170*"33" + 0.168*"1" + 0.158*"10" + 0.147*"4"', '0.399*"36" + 0.364*"10" + -0.295*"31" + 0.245*"20" + -0.226*"38" + 0.194*"26" + 0.194*"15" + 0.194*"39" + 0.194*"23" + 0.194*"40"']


2019-07-23 18:02:08,626 : WARNING : no word id mapping provided; initializing from corpus, assuming identity
2019-07-23 18:02:08,731 : INFO : using serial LSI version on this node
2019-07-23 18:02:08,732 : INFO : updating model with new documents
2019-07-23 18:02:08,775 : INFO : preparing a new chunk of documents
2019-07-23 18:02:08,835 : INFO : using 100 extra samples and 2 power iterations
2019-07-23 18:02:08,836 : INFO : 1st phase: constructing (1000, 104) action matrix
2019-07-23 18:02:08,860 : INFO : orthonormalizing (1000, 104) action matrix
2019-07-23 18:02:08,953 : INFO : 2nd phase: running dense svd on (104, 1033) matrix
2019-07-23 18:02:08,960 : INFO : computing the final decomposition
2019-07-23 18:02:08,961 : INFO : keeping 4 factors (discarding 53.464% of energy spectrum)
2019-07-23 18:02:08,962 : INFO : processed documents up to #1033
2019-07-23 18:02:08,963 : INFO : topic #0(1512.272): -0.672*"360" + -0.232*"767" + -0.207*"189" + -0.182*"890" + -0.144*"697" + -0.136*"752

[(0, '-0.672*"360" + -0.232*"767" + -0.207*"189" + -0.182*"890" + -0.144*"697" + -0.136*"752" + -0.116*"515" + -0.113*"626" + -0.100*"992" + -0.096*"6"'), (1, '-0.456*"360" + 0.201*"779" + 0.186*"81" + 0.167*"800" + 0.156*"230" + 0.140*"992" + -0.135*"697" + 0.115*"139" + 0.109*"462" + 0.105*"744"')]


In [19]:
tokenized_data = [x.split(' ') for x in df_articles['articles']]

In [20]:
dictionary = corpora.Dictionary(tokenized_data)
dictionary.filter_extremes(no_above=0.9)
corpus = [dictionary.doc2bow(text) for text in tokenized_data]
NUM_TOPICS = 5
# Build the LDA model
lda_model = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary1, per_word_topics=True, alpha='asymmetric', minimum_probability=0.0)
topic_distribution = lda_model.show_topics(num_words=10)
pd.DataFrame([x[1].split(' + ') for x in topic_distribution]).T

2019-07-23 18:02:12,487 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-07-23 18:02:13,813 : INFO : built Dictionary(68976 unique tokens: ['', '\n', 'BB', 'Bloomberg', 'Druckenmiller']...) from 1033 documents (total 2434620 corpus positions)
2019-07-23 18:02:13,876 : INFO : discarding 54578 tokens: [('', 1030), ('Druckenmiller', 1), ('Helmut', 1), ('Lamont', 1), ('Norman', 2), ('Schlesinger', 1), ('Stanley', 4), ('一写', 3), ('一家老小', 1), ('下期', 2)]...
2019-07-23 18:02:13,877 : INFO : keeping 14398 tokens which were in no less than 5 and no more than 929 (=90.0%) documents
2019-07-23 18:02:13,892 : INFO : resulting dictionary: Dictionary(14398 unique tokens: ['\n', 'BB', 'Bloomberg', 'X', '一']...)


NameError: name 'dictionary1' is not defined

#### 把文章粗略划分成5个大类

In [ ]:
n_topics = 5
lda = LatentDirichletAllocation(n_components=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)


In [21]:
lda.fit(tf)

NameError: name 'lda' is not defined

####  Build a Latent Semantic Indexing Model

In [20]:
lsi_model = TruncatedSVD(n_components=n_topics)
lsi_Z = lsi_model.fit_transform(tf)
print(lsi_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)

(951, 5)


#### 主题没有一个确定的名称，而是用一系列关键词刻画的。我们定义以下的函数，把每个主题里面的前若干个关键词显示出来：

In [21]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()


In [22]:
n_top_words = 10

In [23]:
tf_feature_names = tf_vectorizer.get_feature_names()

In [31]:
print_top_words(lda, tf_feature_names, n_top_words)


Topic #0:
策略 股票 收益 组合 风险 因子 数据 指数 资产 模型
Topic #1:
基金 公司 管理 私募 机构 资产 产品 信托 业务 企业
Topic #2:
这个 自己 他们 因为 所以 时候 但是 不是 什么 还是
Topic #3:
扑克 策略 宏观 报名 商品 全球 中国 产业 如何 财经
Topic #4:
经济 美国 中国 利率 影响 政策 贸易 预期 企业 增长



#### document-topic distribution

In [32]:
doc_topic_distribution = [[z[1] for z in lda_model[y][0]] for x,y in enumerate(corpus)]

NameError: name 'corpus' is not defined

### visualizing

In [25]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

/home/wuwei/anaconda3/envs/tfgpu/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.014481 -0.055769       1        1  28.476860
2     -0.045012  0.119822       2        1  25.781475
1     -0.059220 -0.178282       3        1  18.560490
0     -0.140579  0.082682       4        1  17.398225
3      0.230330  0.031548       5        1   9.782951, topic_info=    Category         Freq  Term        Total  loglift  logprob
352  Default  4621.000000    基金  4621.000000  30.0000  30.0000
759  Default  2531.000000    策略  2531.000000  29.0000  29.0000
762  Default  2454.000000    管理  2454.000000  28.0000  28.0000
492  Default  1121.000000    扑克  1121.000000  27.0000  27.0000
747  Default  1689.000000    私募  1689.000000  26.0000  26.0000
190  Default  3468.000000    公司  3468.000000  25.0000  25.0000
775  Default  3701.000000    经济  3701.000000  24.0000  24.0000
330  Default  1320.000000    因子  1320.000000  23.0000  23.0000
771  Default  1489.000000    组合  1489.000000  22.0000  22.0000
809  Default  2069.000000    股票  2069.000000  21.0000  21.0000
230  Default  1997.000000    利率  1997.000000  20.0000  20.0000
539  Default  1905.000000    收益  1905.000000  19.0000  19.0000
816  Default  1703.000000    自己  1703.000000  18.0000  18.0000
642  Default  1113.000000    模型  1113.000000  17.0000  17.0000
918  Default  2177.000000    这个  2177.000000  16.0000  16.0000
387  Default  1018.000000    宏观  1018.000000  15.0000  15.0000
796  Default  3354.000000    美国  3354.000000  14.0000  14.0000
78   Default  3676.000000    中国  3676.000000  13.0000  13.0000
505  Default   633.000000    报名   633.000000  12.0000  12.0000
320  Default  1154.000000    商品  1154.000000  11.0000  11.0000
620  Default  1503.000000    机构  1503.000000  10.0000  10.0000
172  Default   890.000000    信托   890.000000   9.0000   9.0000
103  Default   861.000000    产业   861.000000   8.0000   8.0000
105  Default  1345.000000    产品  1345.000000   7.0000   7.0000
663  Default  1388.000000    波动  1388.000000   6.0000   6.0000
124  Default  1396.000000    他们  1396.000000   5.0000   5.0000
71   Default   856.000000    业务   856.000000   4.0000   4.0000
513  Default  2180.000000    指数  2180.000000   3.0000   3.0000
885  Default  1313.000000    贸易  1313.000000   2.0000   2.0000
572  Default  1277.000000    时候  1277.000000   1.0000   1.0000
..       ...          ...   ...          ...      ...      ...
859   Topic5   333.899209    论坛   365.095950   2.2352  -4.9664
0     Topic5   173.219799    AI   195.174799   2.2052  -5.6226
345   Topic5   296.108112    垃圾   335.499841   2.1996  -5.0865
695   Topic5   163.125880    现场   185.329298   2.1969  -5.6827
827   Topic5   146.691626   董事长   167.813447   2.1900  -5.7889
708   Topic5   182.871920    用户   209.306650   2.1895  -5.5684
983   Topic5   173.559169    顶级   207.841516   2.1443  -5.6207
140   Topic5   171.236529    优惠   205.317148   2.1430  -5.6341
698   Topic5   140.063223    现货   168.527061   2.1395  -5.8351
994   Topic5   346.572437    首席   424.799984   2.1210  -4.9291
746   Topic5   217.942520    福利   278.215659   2.0804  -5.3930
103   Topic5   557.554115    产业   861.440561   1.8895  -4.4536
387   Topic5   635.028997    宏观  1018.532258   1.8521  -4.3235
365   Topic5   364.634979    大宗   547.279956   1.9185  -4.8783
320   Topic5   628.410601    商品  1154.982517   1.7159  -4.3340
142   Topic5   354.635756    会议   550.924888   1.8840  -4.9061
617   Topic5   412.642695    机会   805.154051   1.6561  -4.7546
374   Topic5   535.588598    如何  1257.170246   1.4713  -4.4938
812   Topic5   334.862960    背景   571.927760   1.7892  -4.9635
606   Topic5   258.411171  有限公司   384.586953   1.9269  -5.2226
759   Topic5   652.200445    策略  2531.821853   0.9682  -4.2968
185   Topic5   560.828756    全球  1951.962815   1.0774  -4.4478
577   Topic5   365.216175    是否   865.610972   1.4616  -4.8767
45    Topic5   283.837798   下半年   539.848297   1.6816  -5.1288
78    Topic5   558.298707    中国  3676.724385   0.43

In [26]:
data = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

/home/wuwei/anaconda3/envs/tfgpu/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [27]:
pyLDAvis.show(data)


Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]

stopping Server...


### 5. find the documents similarity matrix

In [ ]:
# docu
from sklearn.metrics.pairwise import euclidean_distances
 
def most_similar(x, Z, top_n=5):
    dists = euclidean_distances(x.reshape(1, -1), Z)
    pairs = enumerate(dists[0])
    most_similar = sorted(pairs, key=lambda item: item[1])[:top_n]
    return most_similar
 
similarities = most_similar(x, nmf_Z)
document_id, similarity = similarities[0]
print(data[document_id][:1000])